In [ ]:
import pandas as pd
import numpy as np
import tushare as ts
# 让jupyter正常显示图表
%matplotlib notebook
df = ts.get_k_data('159919', ktype='D', start='2011-19-27', end='2021-10-13')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index(['date'])
df

# 最简单的框架

In [ ]:
import backtrader as bt
import datetime


class GridStrategy(bt.Strategy):

    '''
    网格交易策略'''

    def log(self,text,dt=None):
        '''日志记录函数'''
        date = dt or self.datas[0].datetime.date(0)
        print(f'时间：{date.isoformat()}，日志：{text}')
        
    def __init__(self):
        '''对收盘价整条线的引用'''
        self.dataclose = self.datas[0].close

        '''跟踪待处理的订单（定义一个空订单）'''
        self.order = None
    def notify_order(self,order):
        '''订单的判别 & 处理'''
        if order.status in [order.Submitted, order.Accepted]:  # 顶单在提交~成交的过程中
            return                                             # 无事可做
        elif order.status in [order.Completed]:
            if    order.isbuy(): # 买单
                self.log(f'买单已成交, 成交价格：{order.executed.price}')
                # ！！！记录买单成交的bar！！！
                self.bar_executed = len(self) # ！！！记录买单成交的bar！！！
                
            elif order.issell(): # 卖单
                self.log(f'卖单已成交, 成交价格：{order.executed.price}')

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(('订单被取消 or 订单远离价格无法成交 or 订单被取消'))
        
        # 判断完成后重置订单
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'交易利润：毛【{trade.pnl}】净【{trade.pnlcomm}】')
    
    def next(self):
        

        # 调用日志功能
        self.log(f'收盘价：{self.dataclose[0]}')
        
        # 查看订单跟踪器中是否有订单
        if self.order:
            return
        else:
            pass
        
        # 看看我们是否有仓位
        if not self.position:
            # 还没有建仓 -- 尝试建仓做多
            if self.dataclose[0] < self.dataclose[-1]:
                # 当前收盘价小于前一K线收盘价

                if self.dataclose[-1] < self.dataclose[-2]:
                    # 前一收盘价小于更前一收盘价

                    # 买，买，买!!! (应用所有可能的默认参数)
                    self.log(f'买单创建，当前收盘价：{self.dataclose[0]}')
                    # 下买单并赋值给订单跟踪器 --来避免重复下单
                    self.order = self.buy(size=abs(100))
        else:
            # 已经有仓位 -- 尝试平仓做空
            if len(self) >= (self.bar_executed + 5):
                # 卖，卖，卖!!! (应用所有可能的默认参数)
                self.log(f'卖单创建，当前收盘价：{self.dataclose[0]}')
                # 下卖单并赋值给订单跟踪器 --来避免重复下单
                self.order = self.sell(size=abs(100))

                
            



if __name__ == '__main__':

    # 创建‘大脑’。Cerebro引擎被实例化了
    cerebro = bt.Cerebro()
    
    # 添加一个策略
    cerebro.addstrategy(GridStrategy)
    
    
    # 喂入pandas数据
    data = bt.feeds.PandasData(
        dataname=df,
        # 不接收这个日期更早的数据
        fromdate=datetime.datetime(2012, 1, 31),
        # 不接收晚于这个日期的数据
        todate=datetime.datetime(2021, 10, 15)
        )
    
    
    # 把数据槽添加到大脑引擎中
    cerebro.adddata(data)
    
    
    # 创建初始金额
    cerebro.broker.setcash(50000.0)
    
    
    # 设定佣金为万0.5，0.5除以10000
    cerebro.broker.setcommission(commission=0.00005)
    
    
    print('策略执行前的资金: %.2f' % cerebro.broker.getvalue())
    
    
    # ‘大脑’运行。生成的cerebro实例被告知运行（遍历数据）
    cerebro.run()
    
    
    print('策略执行后的资金: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()